<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/fdcir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 25.3 MB/s eta 0:00:00


In [ ]:
import QuantLib as ql
import unittest
import math # For math.fabs, though assertAlmostEqual handles this

class FdCIRTests(unittest.TestCase):

    def testFdmCIRConvergence(self):
        """Testing FDM CIR convergence."""
        print("Testing FDM CIR convergence...")

        # Original C++ schemes:
        # FdmSchemeDesc schemes[] = {
        #     FdmSchemeDesc::Hundsdorfer(),
        #     FdmSchemeDesc::ModifiedCraigSneyd(),
        #     FdmSchemeDesc::ModifiedHundsdorfer(),
        #     FdmSchemeDesc::CraigSneyd(),
        #     FdmSchemeDesc::TrBDF2(),
        #     FdmSchemeDesc::CrankNicolson(),
        # };
        schemes_py = [
            ql.FdmSchemeDesc.Hundsdorfer(),
            ql.FdmSchemeDesc.ModifiedCraigSneyd(),
            ql.FdmSchemeDesc.ModifiedHundsdorfer(),
            ql.FdmSchemeDesc.CraigSneyd(),
            ql.FdmSchemeDesc.TrBDF2(),
            ql.FdmSchemeDesc.CrankNicolson(),
        ]

        # Set up dates
        # Use a fixed date for reproducibility, similar to what TopLevelFixture might do.
        # Date today = Date::todaysDate();
        # Let's pick a fixed date
        eval_date = ql.Date(15, ql.May, 2020)
        ql.Settings.instance().evaluationDate = eval_date
        today = ql.Settings.instance().evaluationDate

        # Option parameters
        option_type = ql.Option.Put
        underlying_s0 = 36.0
        strike_price = 40.0
        dividend_yield = 0.00
        risk_free_rate = 0.06
        volatility = 0.20
        # maturity = today + 365;
        maturity_date = today + ql.Period(365, ql.Days) # More robust way
        day_counter = ql.Actual365Fixed()

        # ext::shared_ptr<Exercise> europeanExercise(new EuropeanExercise(maturity));
        european_exercise = ql.EuropeanExercise(maturity_date)

        # Handle<Quote> underlyingH(ext::shared_ptr<Quote>(new SimpleQuote(underlying)));
        underlying_h = ql.QuoteHandle(ql.SimpleQuote(underlying_s0))

        # Handle<YieldTermStructure> flatTermStructure(
        #     ext::shared_ptr<YieldTermStructure>(flatRate(today, riskFreeRate, dayCounter)));
        flat_term_structure_h = ql.YieldTermStructureHandle(
            ql.FlatForward(today, risk_free_rate, day_counter)
        )
        # Handle<YieldTermStructure> flatDividendTS(
        #     ext::shared_ptr<YieldTermStructure>(flatRate(today, dividendYield, dayCounter)));
        flat_dividend_ts_h = ql.YieldTermStructureHandle(
            ql.FlatForward(today, dividend_yield, day_counter)
        )
        # Handle<BlackVolTermStructure> flatVolTS(
        #     ext::shared_ptr<BlackVolTermStructure>(flatVol(today, volatility, dayCounter)));
        # flatVol typically creates a BlackConstantVol. Calendar is needed.
        flat_vol_ts_h = ql.BlackVolTermStructureHandle(
            ql.BlackConstantVol(today, ql.NullCalendar(), volatility, day_counter)
        )

        # ext::shared_ptr<StrikedTypePayoff> payoff(new PlainVanillaPayoff(type, strike));
        payoff = ql.PlainVanillaPayoff(option_type, strike_price)

        # ext::shared_ptr<BlackScholesMertonProcess> bsmProcess(
        #     new BlackScholesMertonProcess(underlyingH, flatDividendTS,
        #                                   flatTermStructure, flatVolTS));
        bsm_process = ql.BlackScholesMertonProcess(
            underlying_h, flat_dividend_ts_h, flat_term_structure_h, flat_vol_ts_h
        )

        # VanillaOption europeanOption(payoff, europeanExercise);
        european_option = ql.VanillaOption(payoff, european_exercise)

        # CIR parameters
        speed = 1.2188        # kappa (mean reversion speed of variance)
        cir_sigma = 0.02438   # sigma_v (volatility of variance)
        level = 0.0183        # theta (long-term mean of variance under P-measure or similar)
        initial_rate = 0.06   # v0 (initial variance) - C++ name "initialRate" is for CIR process state
        rho = 0.00789         # correlation between asset and variance processes
        lambda_param = -0.5726  # market price of volatility risk parameter

        # Transformation for CIR process parameters (often for risk-neutral measure)
        # newSpeed = speed + (cirSigma*lambda); //kappa* = kappa - lambda*sigma_v (if lambda is market price of vol risk)
                                              # Here, it seems to be kappa_star = kappa_P + lambda_v * sigma_v (as in Heston)
        new_speed = speed + (cir_sigma * lambda_param)
        # newLevel = (level * speed)/(speed + (cirSigma*lambda)); // theta* = (kappa_P * theta_P) / kappa*
        new_level = (level * speed) / new_speed if new_speed != 0 else 0 # Avoid division by zero

        # ext::shared_ptr<CoxIngersollRossProcess> cirProcess(
        #    new CoxIngersollRossProcess(newSpeed, cirSigma, initialRate, newLevel));
        # QL CIRProcess: speed (kappa), vol (sigma_v), x0 (initial value), level (theta)
        cir_process = ql.CoxIngersollRossProcess(
            new_speed,    # speed (kappa*)
            cir_sigma,    # vol (sigma_v)
            initial_rate, # x0 (initial variance v0)
            new_level     # level (theta*)
        )

        expected_npv = 4.275
        tolerance = 0.0003

        # Default grid sizes from C++ MakeFdCIRVanillaEngine if not specified:
        # tGrid = 100, xGrid = 100, vGrid = 50, dampingSteps = 0
        t_grid = 100
        x_grid = 100
        v_grid = 50 # Grid for the variance (CIR) process
        damping_steps = 0

        for scheme_desc in schemes_py:
            # C++: ext::shared_ptr<PricingEngine> fdcirengine =
            #         MakeFdCIRVanillaEngine(cirProcess, bsmProcess, rho).withFdmSchemeDesc(scheme);
            # The MakeFdCIRVanillaEngine helper in C++ constructs the engine and then applies the scheme.
            # In Python, we can pass the scheme directly to the constructor if available,
            # or use a setter method.
            # FdCIRVanillaEngine(cirProcess, bsmProcess, rho, tGrid, xGrid, vGrid, dampingSteps, schemeDesc)

            fd_cir_engine = ql.FdCIRVanillaEngine(
                cir_process,
                bsm_process, # This is the process for the underlying asset price
                rho,
                t_grid,
                x_grid,
                v_grid,
                damping_steps,
                scheme_desc # Pass the scheme description here
            )

            european_option.setPricingEngine(fd_cir_engine)
            calculated_npv = european_option.NPV()

            self.assertAlmostEqual(calculated_npv, expected_npv, delta=tolerance,
                                   msg=(f"Failed for scheme: {scheme_desc.type()}\n"
                                        f"    calculated NPV: {calculated_npv:.5f}\n"
                                        f"    expected NPV:   {expected_npv:.5f}\n"
                                        f"    diff:           {abs(calculated_npv - expected_npv):.5e}\n"
                                        f"    tolerance:      {tolerance}"))
            # print(f"Scheme: {scheme_desc.type()}, NPV: {calculated_npv:.5f}") # Optional: for debugging

if __name__ == '__main__':
    print("C++ test suite: FdCIRTests")
    print("Python QuantLib version: " + ql.__version__)
    print("-------------------------------------------------------")
    suite = unittest.TestSuite()
    suite.addTest(unittest.makeSuite(FdCIRTests))
    unittest.TextTestRunner(verbosity=2).run(suite)

Key changes and considerations:
unittest Framework: Standard Python unittest module is used.
Date Handling:
ql.Date.todaysDate() is replaced by a fixed date (ql.Date(15, ql.May, 2020)) and set via ql.Settings.instance().evaluationDate. This is crucial for test reproducibility, as TopLevelFixture in C++ likely does something similar.
Maturity calculation today + 365 is changed to today + ql.Period(365, ql.Days) which is slightly more robust and idiomatic in QuantLib.
Handles:
Handle<Quote> becomes ql.QuoteHandle(ql.SimpleQuote(...)).
Handle<YieldTermStructure> becomes ql.YieldTermStructureHandle(ql.FlatForward(...)).
Handle<BlackVolTermStructure> becomes ql.BlackVolTermStructureHandle(ql.BlackConstantVol(...)). Note that the C++ flatVol helper typically creates a BlackConstantVol. The Python ql.BlackConstantVol constructor needs a calendar (here, ql.NullCalendar() for a generic flat vol) and the day counter.
Object Creation:
C++ ext::shared_ptr<Type>(new Type(...)) maps to Python ql.Type(...).
Option::Type type(Option::Put) maps to option_type = ql.Option.Put.
lambda Keyword: The C++ variable lambda is renamed to lambda_param in Python to avoid clashing with Python's lambda keyword.
CoxIngersollRossProcess Parameters:
The C++ code uses newSpeed, cirSigma, initialRate, newLevel.
The QuantLib Python CoxIngersollRossProcess takes (speed, vol, x0, level).
new_speed maps to speed (mean reversion speed, kappa).
cir_sigma maps to vol (volatility of the variance process, sigma_v).
initial_rate (from C++) maps to x0 (initial value of the process, v0).
new_level maps to level (long-term mean of the process, theta).
The initialRate variable in the C++ code (value 0.06) refers to the initial value of the CIR process, which in this context models the variance (v0).
FdCIRVanillaEngine Instantiation:
The C++ code uses MakeFdCIRVanillaEngine(...).withFdmSchemeDesc(scheme).
The MakeFdCIRVanillaEngine is a C++ helper function. In Python, you typically construct the engine directly.
The FdCIRVanillaEngine constructor in Python directly accepts the FdmSchemeDesc as an argument, along with grid parameters. I've used the default grid sizes (t_grid, x_grid, v_grid, damping_steps) that are implicitly used by the C++ MakeFdCIRVanillaEngine if not specified.
Assertions: BOOST_ERROR is replaced by self.assertAlmostEqual(calculated, expected, delta=tolerance, msg=...). The msg parameter provides a detailed error message similar to the C++ version.
Scheme Iteration: The loop over schemes is directly translated. scheme_desc.type() is used to get a string representation of the FDM scheme type for better error messages.
Clarity of Variable Names: Some variable names were made slightly more descriptive in Python (e.g., underlying_s0, strike_price, maturity_date).
Division by Zero: Added a check for new_speed != 0 before division when calculating new_level to prevent potential division by zero errors, though in this specific case new_speed is unlikely to be zero with the given parameters